### 0. Imports necesarios

In [8]:
import bootcampviztools as bt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split, GridSearchCV
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, classification_report
from sklearn.base import BaseEstimator, TransformerMixin

import lightgbm as lgb
import os
import dill



### 1. Carga de datos de Test e identificación de columnas

In [9]:
df_test = pd.read_csv('../data/test.csv', index_col=0)

# Ponemos 'id' como índice
df_test = df_test.set_index('id')

In [3]:
'''
#Identificar tipos de columnas
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
binary_cols = [col for col in df_test.select_dtypes(include=['object']).columns if df_test[col].nunique() == 2]

# Quitar el target
binary_cols.remove('satisfaction')

# Establecer el target
target = 'satisfaction'


onehot_cols = [col for col in df_test.columns if col not in numerical_cols and col not in binary_cols and col not in target]


#features numericas a las que aplicar logaritmo:
numerical_cols_log = [col for col in numerical_cols if col != 'Age']'
''''

SyntaxError: unterminated string literal (detected at line 18) (2120849580.py, line 18)

### 2. Cargamos el modelo

In [24]:
print(best_pipeline)

Pipeline(steps=[('null_handling',
                 Pipeline(steps=[('drop_high_nulls', DropHighNullColumns()),
                                 ('drop_remaining_nulls', DropRemainingNulls()),
                                 ('drop_duplicates', DropDuplicates())])),
                ('column_identification', IdentifyColumns()),
                ('log_transform',
                 FunctionTransformer(func=<function safe_log_transform at 0x0000022D95E928E0>)),
                ('preprocessor',
                 Pipeline(steps=[('pr...
                                                                    'boarding',
                                                                    'Seat '
                                                                    'comfort',
                                                                    'Inflight '
                                                                    'entertainment',
                                                                  

In [59]:
import os
import dill
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report

# ✅ Definir las rutas basadas en la estructura del directorio
BASE_PATH = "../src"  # Ajustado a la estructura vista en la imagen
MODEL_PATH = os.path.join(BASE_PATH, "src/models/best_pipeline.pkl")
TEST_PATH = os.path.join(BASE_PATH, "data/test.csv")
PREDICTIONS_PATH = os.path.join(BASE_PATH, "data/predictions.csv")

# ✅ Cargar el pipeline entrenado
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"El archivo del modelo no se encontró en: {MODEL_PATH}")

with open(MODEL_PATH, "rb") as f:
    best_pipeline = dill.load(f)

print("✅ Modelo y preprocesador cargados exitosamente.")

# ✅ Cargar los datos de test
if not os.path.exists(TEST_PATH):
    raise FileNotFoundError(f"No se encontró el archivo de test en {TEST_PATH}")

df_test = pd.read_csv(TEST_PATH, index_col=0)  # `id` ya es índice

# ✅ Extraer la variable target si está presente
if "satisfaction" in df_test.columns:
    y_test = df_test["satisfaction"].replace({"neutral or dissatisfied": 0, "satisfied": 1}).infer_objects(copy=False)
    df_test = df_test.drop(columns=["satisfaction"])
else:
    y_test = None  # Si no está presente, asumimos que se predice sin evaluación

# ✅ Aplicar el pipeline completo al conjunto de test (preprocesamiento + modelo)
y_pred = best_pipeline.predict(df_test)

# ✅ Evaluar el modelo si se tiene la columna 'satisfaction' en test
if y_test is not None:
    acc = accuracy_score(y_test, y_pred)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)

    print("\n📊 Métricas de Evaluación:")
    print(f"Accuracy: {acc:.4f}")
    print(f"Balanced Accuracy: {bal_acc:.4f}")
    print("\nReporte de Clasificación:\n", class_report)

# ✅ Guardar las predicciones en la estructura correcta
predictions_df = pd.DataFrame({"id": df_test.index, "prediction": y_pred})
predictions_df.to_csv(PREDICTIONS_PATH, index=False)

print(f"\n✅ Predicciones guardadas en {PREDICTIONS_PATH}")


FileNotFoundError: El archivo del modelo no se encontró en: ../src\src/models/best_pipeline.pkl

### 3. Aplicamos los pipelines

In [ ]:
# Crear un pipeline que incluya el preprocesamiento antes del modelo
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),  # Asegurar que el preprocesador se aplica a test
    ("model", grid_search.best_estimator_)  # Modelo final con los mejores hiperparÃ¡metros
])

# Definir la ruta correcta para guardar el pipeline completo
MODEL_DIR = os.path.join(os.getcwd(), "..", "..", "src", "models")
os.makedirs(MODEL_DIR, exist_ok=True)

model_path = os.path.join(MODEL_DIR, "best_pipeline.pkl")

# Guardar el pipeline completo
with open(model_path, "wb") as f:
    pickle.dump(full_pipeline, f)

print(f"Pipeline completo guardado en {model_path}")

NameError: name 'preprocessor' is not defined

In [ ]:
# Cargar el pipeline entrenado
print(f"Cargando pipeline desde {model_path}...")
with open(model_path, "rb") as f:
    pipeline = pickle.load(f)

print("Pipeline cargado correctamente.")

# Separar características y variable objetivo
X_test = df_test.drop(columns=["satisfaction"])
y_test = df_test["satisfaction"]

# Aplicar el pipeline y realizar predicciones
print("Generando predicciones en los datos de test...")
y_pred = pipeline.predict(X_test)

Cargando pipeline desde c:\Users\maria\Documents\GitHub\Pipelines_Airline_Passenger_Satisfaction\src\models\LightGBM_optimized_pipeline.pkl...
Pipeline cargado correctamente.
Generando predicciones en los datos de test...


ValueError: Number of features of the model must match the input. Model n_features_ is 93 and input n_features is 22

#### 4. Métricas de evaluación

In [ ]:
# Classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Convertir el reporte en un DataFrame
report_df = pd.DataFrame(report).T  # Transponer para mejor visualización

# Mostrar el classification report como tabla
report_df

La primera métrica escogida se trata de la matriz de confusión ya que nos permite tener una visión de varias métricas a la vez, además de percibir si ha habido sobreajuste de datos.

Tal y como se observa, nuestro modelo optimizado optiene un XX% de accuracy

In [ ]:
# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")
plt.title(f"Matriz de Confusión - {best_model_name}")
plt.show()

In [ ]:
# Importancia de características (solo para modelos de árboles)
if best_model_name in ["RandomForest", "GradientBoosting", "LightGBM"]:
    importances = best_model.named_steps["model"].feature_importances_
    feature_names = X_train.columns

    # Crear un DataFrame ordenado por importancia
    feature_importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
    feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False).head(20)  # Mostrar solo las 20 más importantes

    # Graficar la importancia de características
    plt.figure(figsize=(12, 8))
    sns.barplot(x=feature_importance_df["Importance"], y=feature_importance_df["Feature"], palette="Blues_r")

    # Etiquetas y título
    plt.title(f"Importancia de Características - {best_model_name}", fontsize=14)
    plt.xlabel("Importancia", fontsize=12)
    plt.ylabel("Características", fontsize=12)

    # Ajustar etiquetas para mejor legibilidad
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    # Mostrar valores sobre las barras
    for index, value in enumerate(feature_importance_df["Importance"]):
        plt.text(value + 2, index, f"{value:.2f}", fontsize=10, verticalalignment="center")

    plt.show()